In [81]:
train_data_v2 = np.load('training_data_v2.npy', allow_pickle=True)

In [80]:
from keras_preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255.,
                             validation_split=0.2,
                             rotation_range=10, # rotation
                             width_shift_range=0.2, # horizontal shift
                             height_shift_range=0.2, # vertical shift
                             zoom_range=0.2, # zoom
                             horizontal_flip=True, # horizontal flip
                             brightness_range=[0.2,1.2]) # brightness)

In [ ]:
train_generator  = datagen.flow_from_dataframe(
    dataframe=train_data_v2[1],
    directory=save_dir,
    x_col="path",
    y_col="binary_score",
    target_size=(80, 60),
    color_mode="rgb",
    batch_size=64,
    save_to_dir='png_images_resized/',
    save_format="png",
    class_mode="binary",
    subset='training',
    shuffle = True
)


validation_generator  = datagen.flow_from_dataframe(
    dataframe=train_clean,
    directory=save_dir,
    x_col="path",
    y_col="binary_score",
    target_size=(258, 258),
    color_mode="rgb",
    batch_size=64, #deberia ser mas alto que el del modelo (sugerencia)
    save_to_dir='png_images_resized/',
    save_format="png",
    class_mode="binary",
    subset='validation',
    shuffle = True
)

In [74]:
# import tflearn
# from tflearn.layers.conv import conv_2d, max_pool_2d
# from tflearn.layers.core import input_data, dropout, fully_connected
# from tflearn.layers.estimator import regression
# from tflearn.layers.normalization import local_response_normalization
import tensorflow as tf
import numpy as np

def alexnet(width, height, lr):
    network = input_data(shape=[None, width, height, 1], name='input')
    network = conv_2d(network, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 3, activation='softmax')
    network = regression(network, optimizer='momentum',
                         loss='categorical_crossentropy',
                         learning_rate=lr, name='targets')

    model = tflearn.DNN(network, checkpoint_path='model_alexnet',
                        max_checkpoints=1, tensorboard_verbose=2)

    return model

In [75]:
WIDTH = 80
HEIGHT = 60
LR = 1e-3
EPOCHS = 8
MODEL_NAME = 'pygta5-car-{}-{}-{}-epochs.model'.format(LR, 'alexnetv2',EPOCHS)

model = alexnet(WIDTH, HEIGHT, LR)

In [76]:
train_data = np.load('training_data_v2.npy', allow_pickle=True)


train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,WIDTH,HEIGHT,1)
Y = np.array([i[1] for i in train])

test_x = np.array([i[0] for i in test]).reshape(-1,WIDTH,HEIGHT,1)
test_y = np.array([i[1] for i in test])

In [77]:
model.fit(X, Y, n_epoch=EPOCHS, validation_set= (test_x, test_y), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

IndexError: list index out of range

In [ ]:
model.save(MODEL_NAME)

In [5]:
import tensorflow as tf

In [6]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=4)

In [13]:
model = tf.keras.Sequential()


model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(80, 60, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(64, activation="relu"))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(32, activation="relu"))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(16, activation="relu"))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [14]:
model.compile(optimizer='adam', 
              loss="binary_crossentropy", 
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 78, 58, 16)        160       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 39, 29, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 37, 27, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 18, 13, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 18, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 5, 64)         

In [16]:
WIDTH = 80
HEIGHT = 60
LR = 1e-3
EPOCHS = 8
MODEL_NAME = 'pygta5-car-{}-{}-{}-epochs.model'.format(LR, 'alexnetv2',EPOCHS)

In [17]:
train_data = np.load('training_data_v2.npy', allow_pickle=True)

train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,WIDTH,HEIGHT,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,WIDTH,HEIGHT,1)
test_y = [i[1] for i in test]

In [20]:
history = model.fit(
    x = X,
    y = Y,
    validation_data = (test_x, test_y),
    batch_size=32,
    epochs = EPOCHS,
    callbacks=[callback])

ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), for inputs ['dense_7'] but instead got the following list of 18322 arrays: [array([[1],
       [0],
       [0]]), array([[1],
       [0],
       [0]]), array([[1],
       [0],
       [0]]), array([[1],
       [0],
       [0]]), array([[0],
       [0],
       [1]]), array([[0...